# Introduction

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        path = os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head()

**pH value**<br>
PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status. WHO has recommended maximum permissible limit of pH from 6.5 to 8.5. The current investigation ranges were 6.52–6.83 which are in the range of WHO standards.

**Hardness**<br>
Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.

**Solids (Total dissolved solids - TDS)**<br>
Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced un-wanted taste and diluted color in appearance of water. This is the important parameter for the use of water. The water with high TDS value indicates that water is highly mineralized. Desirable limit for TDS is 500 mg/l and maximum limit is 1000 mg/l which prescribed for drinking purpose.

**Chloramines**<br>
Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water. Chlorine levels up to 4 milligrams per liter (mg/L or 4 parts per million (ppm)) are considered safe in drinking water.

**Sulfate**<br>
Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food. The principal commercial use of sulfate is in the chemical industry. Sulfate concentration in seawater is about 2,700 milligrams per liter (mg/L). It ranges from 3 to 30 mg/L in most freshwater supplies, although much higher concentrations (1000 mg/L) are found in some geographic locations.

**Conductivity**<br>
Pure water is not a good conductor of electric current rather’s a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity. Electrical conductivity (EC) actually measures the ionic process of a solution that enables it to transmit current. According to WHO standards, EC value should not exceeded 400 μS/cm.

**Organic_carbon**<br>
Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water. According to US EPA < 2 mg/L as TOC in treated / drinking water, and < 4 mg/Lit in source water which is use for treatment.

**Trihalomethanes**<br>
THMs are chemicals which may be found in water treated with chlorine. The concentration of THMs in drinking water varies according to the level of organic material in the water, the amount of chlorine required to treat the water, and the temperature of the water that is being treated. THM levels up to 80 ppm is considered safe in drinking water.

**Turbidity**<br>
The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter. The mean turbidity value obtained for Wondo Genet Campus (0.98 NTU) is lower than the WHO recommended value of 5.00 NTU.

**Potability**<br>
Indicates if water is safe for human consumption where 1 means Potable and 0 means Not potable.

In [ ]:
df['Potability'].value_counts().plot.pie(autopct='%.2f%%', 
                                         labels=['Not potable', 'Potable'], 
                                         ylabel='', 
                                         title='Potability')
plt.show()

We see that 61% of observations represent water that is not drinkable.

In [ ]:
X_columns = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity']

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(12, 10))

for i in range(3):
  for j in range(3):
    sns.histplot(data=df, x=X_columns[3*i+j], hue='Potability', linewidth=0, ax=ax[i][j])

fig.suptitle('Distribution of features', fontsize=14)
fig.show()

According to the distribution histograms, it can be said that there are no obvious differences between drinking and non-drinking water.

In [ ]:
pdplot = pd.plotting.scatter_matrix(df.fillna(df.mean())[X_columns],
                                    c=df.fillna(df.mean())['Potability'],
                                    s=30,
                                    figsize=(10,10))

The feature matrix shows that the data is linearly inseparable.

# Predictive model

As a predictive model, we will use the K-nearest neighbors method and logistic regression. Logistic regression and nearest neighbors are poorly suited for such a set. Let us verify this and compare the results.

Let's prepare the data for building models.

In [ ]:
df.columns

In [ ]:
X_columns = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity',
       'Organic_carbon', 'Trihalomethanes', 'Turbidity']

In [ ]:
#data has 0, we replace it with mean values
X = df[X_columns].fillna(df[X_columns].mean()).values
y = df['Potability'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

## K-nearest neighbors method

In [ ]:
knn = KNeighborsClassifier()

Let's build a learning curve to find the number K of the nearest neighbors.

In [ ]:
test_scores = []
train_scores = []
n_neighbors = []

for i in range(2, 50):
  knn = KNeighborsClassifier(n_neighbors=i)
  knn.fit(X_train, y_train)
  test_scores.append(knn.score(X_test, y_test))
  train_scores.append(knn.score(X_train, y_train))
  n_neighbors.append(i)

In [ ]:
sns.lineplot(x=n_neighbors, y=test_scores, label='Test scores')
ax = sns.lineplot(x=n_neighbors, y=train_scores, label='Train scores')
ax.set(xlabel='n_neighbors', ylabel='scores')
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)

In [ ]:
print(f'Test score: {knn.score(X_test, y_test)}')
print(f'Train score: {knn.score(X_train, y_train)}')

## Logistic regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
logr = LogisticRegression()
logr.fit(X_train, y_train)

In [ ]:
print(f'Test score: {logr.score(X_test, y_test)}')
print(f'Train score: {logr.score(X_train, y_train)}')

## Comparison of models

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

potability=['Not potable', 'Potable']

plot_confusion_matrix(logr, X_test, y_test, display_labels=potability, values_format='d', ax=ax[0])
ax[0].set_title('LogisticRegression')

plot_confusion_matrix(knn, X_test, y_test, display_labels=potability, values_format='d', ax=ax[1])
ax[1].set_title('KNeighborsClassifier')

fig.show()

In [ ]:
y_pred = logr.predict(X_test)
print(f'LogisticRegression f1 score: {f1_score(y_pred, y_test)}')

y_pred = knn.predict(X_test)
print(f'KNeighborsClassifier f1 score: {f1_score(y_pred, y_test)}')

The logistic regression model gives similar results compared to the KNN model, but f1_score shows that the logistic regression does not work. The model always gives negative predictions, and given that 60% of observations have negative marks, in 60 cases out of 100 the model will be right always.

# Random forest

Since the features in our dataset are continuous values for which you can specify boundary values, this data is well suited for decision trees. Therefore, consider how a random forest will cope with the classification task.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
rfc = RandomForestClassifier(max_depth=10, n_estimators=3000)
rfc.fit(X_train, y_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
potability=['Not potable', 'Potable']
plot_confusion_matrix(rfc, X_test, y_test, display_labels=potability, values_format='d')

In [ ]:
y_pred = rfc.predict(X_test)
print(f'f1 score: {f1_score(y_pred, y_test)}')

In [ ]:
print(f'Test score: {rfc.score(X_test, y_test)}')
print(f'Train score: {rfc.score(X_train, y_train)}')

So, the expectedly random forest shows good results in comparison with the previous models. Here, the random forest is not retrained and generalizes the data well (this is indicated by the estimates on the test and training data).

In [ ]:
plt.barh(X_columns, rfc.feature_importances_)
plt.show()

The defining features for this model are ph and Sulfate.